## Table of Contents

<ul>
    <li><a href='#Dependencies'>Dependencies</a></li>
    </li>
    <li><a href='#Modelling'>Modelling</a>
        <ol style='margin-top: 0em;'>
            <li><a href='#1.-State-Variables'>State Variables</a></li>
            <li><a href='#2.-System-Parameters'>System Parameters</a></li>
            <li><a href='#3.-Policy-Functions'>Policy Functions</a></li>
            <li><a href='#4.-State-Update-Functions'>State Update Functions</a></li>
            <li><a href='#5.-Partial-State-Update-Blocks'>Partial State Update Blocks</a></li>
        </ol>
    </li>
    <li><a href='#Simulation'>Simulation</a>
        <ol style='margin-top: 0em;' start="6">
            <li><a href='#6.-Configuration'>Configuration</a></li>
            <li><a href='#7.-Execution'>Execution</a></li>
            <li><a href='#8.-Output-Preparation'>Output Preparation</a></li>
            <li><a href='#9.-Analysis'>Analysis</a></li>
        </ol>
    </li>
</ul>

# Dependencies

In [1]:
# radCAD standard dependencies

# radCAD Model module
from radcad import Model, Simulation, Experiment

# radCAD simulation engine modules
from radcad.engine import Engine, Backend

# radCAD core method used for post-processing
from radcad.core import generate_parameter_sweep

# Included with radCAD
import pandas as pd

ModuleNotFoundError: No module named 'radcad'

In [ ]:
# Additional dependencies

# For analytics
import numpy as np
# For visualization
import plotly.express as px

# Modelling

## 1. State Variables

In [ ]:
initial_state = {
    'predator_population': 15, # Initial number of predators
    'prey_population': 100, # Initial number of preys
}
initial_state

## 2. System Parameters

In [ ]:
system_params = {
    # Parameters describing the interaction between the two populations:
    
    # A parameter used to calculate the rate of predator birth
    "predator_birth_parameter": [0.01], 
    # A parameter used to calculate the rate of predator death
    "predator_death_parameter": [1.0],
    # A parameter used to calculate the rate of prey birth
    "prey_birth_parameter": [0.6, 1.0],
    # A parameter used to calculate the rate of prey death
    "prey_death_parameter": [0.03],

    # Parameters used for random Numpy variable tuning
    # These parameters scale the random variable used to create the random birth / death rate
    
    "random_predator_birth": [0.0002],
    "random_predator_death": [0.005],
    "random_prey_birth": [0.1],
    "random_prey_death": [0.1],

    # Parameter used as conversion factor between 1 unit of time and 1 timestep
    # 10 timesteps == 1 unit of time, i.e. every 10 cadCAD model timesteps, 1 unit of actual model time passes
    
    "dt": [0.1],
}

## 3. Policy Functions

In [ ]:
def calculate_population(rate: float,
                         population_1: float,
                         population_2: float,
                         dt: float) -> float:
    
    return rate * population_1 * population_2 * dt


def p_predator_births(params, substep, state_history, previous_state):
    '''Predator Births Policy Function
    The predator birth rate (rate of predators born per unit of time) is a product of
    the prey population and the predator birth parameter plus a random variable.
    
    i.e. the larger the prey population, the higher the predator birth rate
    '''
    # Parameters   
    dt = params['dt']
    predator_birth_parameter = params['predator_birth_parameter']
    random_predator_birth = params['random_predator_birth']

    # State Variables
    predator_population = previous_state['predator_population']
    prey_population = previous_state['prey_population']

    # Calculate the predator birth rate
    birth_rate = prey_population * (predator_birth_parameter + np.random.random() * random_predator_birth)
    # Calculate change in predator population
    births = calculate_population(
        birth_rate,
        predator_population,
        1,
        dt
    )

    return {'add_to_predator_population': births}


def p_predator_deaths(params, substep, state_history, previous_state):
    '''Predator Deaths Policy Function
    The predator death rate (rate of predators that die per unit of time) is a function of
    the predator death parameter plus a random variable.
    
    i.e. the larger the predator death parameter, the higher the predator death rate
    '''
    # Parameters
    dt = params['dt']
    predator_death_parameter = params['predator_death_parameter']
    random_predator_death = params['random_predator_death']

    # State Variables
    predator_population = previous_state['predator_population']

    # Calculate the predator death rate
    death_rate = predator_death_parameter + np.random.random() * random_predator_death
    # Calculate change in predator population
    deaths = calculate_population(
        death_rate,
        predator_population,
        1,
        dt
    )

    return {'add_to_predator_population': -1.0 * deaths}


def p_prey_births(params, substep, state_history, previous_state):
    '''Prey Births Policy Function
    The prey birth rate (rate of preys born per unit of time) is a function of
    the prey birth parameter plus a random variable.
    
    i.e. the larger the prey birth parameter, the higher the prey birth rate
    '''
    # Parameters
    dt = params['dt']
    prey_birth_parameter = params['prey_birth_parameter']
    random_prey_birth = params['random_prey_birth']

    # State Variables
    prey_population = previous_state['prey_population']

    # Calculate the prey birth rate
    birth_rate = prey_birth_parameter + np.random.random() * random_prey_birth
    # Calculate change in prey population
    births = calculate_population(
        birth_rate,
        1,
        prey_population,
        dt
    )

    return {'add_to_prey_population': births}


def p_prey_deaths(params, substep, state_history, previous_state):
    '''Prey Deaths Policy Function
    The prey death rate (rate of preys that die per unit of time) is a product of
    the predator population and the prey death parameter plus a random variable.
    
    i.e. the larger the predator population, the higher the prey death rate
    '''
    # Parameters
    dt = params['dt']
    prey_death_parameter = params['prey_death_parameter']
    random_prey_death = params['random_prey_death']

    # State Variables
    prey_population = previous_state['prey_population']
    predator_population = previous_state['predator_population']

    # Calculate the prey death rate
    death_rate = predator_population * (prey_death_parameter + np.random.random() * random_prey_death)
    # Calculate change in prey population
    deaths = calculate_population(
        death_rate,
        1,
        prey_population,
        dt
    )

    return {'add_to_prey_population': -1.0 * deaths}

## 4. State Update Functions

In [ ]:
def s_predator_population(params, substep, state_history, previous_state, policy_input):
    '''Predator Population State Update Function
    Take the Policy Input `add_to_predator_population`
    (the net predator births and deaths)
    and add to the `predator_population` State Variable.
    '''
    # Policy Inputs
    add_to_predator_population = policy_input['add_to_predator_population']

    # State Variables
    predator_population = previous_state['predator_population']

    # Calculate updated predator population
    updated_predator_population = predator_population + add_to_predator_population

    return 'predator_population', updated_predator_population


def s_prey_population(params, substep, state_history, previous_state, policy_input):
    '''Prey Population State Update Function
    Take the Policy Input `add_to_prey_population`
    (the net prey births and deaths)
    and add to the `prey_population` State Variable.
    '''
    # Policy Inputs
    add_to_prey_population = policy_input['add_to_prey_population']

    # State Variables
    prey_population = previous_state['prey_population']

    # Calculate updated prey population
    updated_prey_population = prey_population + add_to_prey_population

    return 'prey_population', updated_prey_population

## 5. Partial State Update Blocks

In [ ]:
partial_state_update_blocks = [
    {   
        # Configure the model Policy Functions
        'policies': {
            # Calculate the predator birth rate and number of births
            'predator_births': p_predator_births,
            # Calculate the predator death rate and number of deaths
            'predator_deaths': p_predator_deaths,
            # Calculate the prey birth rate and number of births
            'prey_births': p_prey_births,
            # Calculate the prey death rate and number of deaths
            'prey_deaths': p_prey_deaths,
        },
        # Configure the model State Update Functions
        'variables': {
            # Update the predator population
            'prey_population': s_prey_population,
            # Update the prey population
            'predator_population': s_predator_population
        }
    }
]

# Simulation

## 6. Configuration

In [ ]:
model = Model(
    initial_state=initial_state,
    state_update_blocks=partial_state_update_blocks,
    params=system_params
)

In [ ]:
simulation = Simulation(model=model, timesteps=400, runs=5)
experiment = Experiment(simulation)

## 7. Execution

In [ ]:
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

raw_result = experiment.run()

## 8. Output Preparation

In [ ]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

# Get a list of all System Parameters
set_params = list(system_params.keys())

# Insert cadCAD parameters for each configuration into DataFrame
if set_params:
    # Get a list of all parameters configurations
    parameter_sweep = generate_parameter_sweep(system_params)
    # Filter only the parameters set in set_params
    parameter_sweep = [{param: subset[param] for param in set_params} for subset in parameter_sweep]
    
    # for each subset of configuration
    for subset_index in df['subset'].unique():
        # For each parameter key value pair
        for (key, value) in parameter_sweep[subset_index].items():
            # Select all DataFrame indices where subset == subset_index
            dataframe_indices = df.eval(f'subset == {subset_index}')
            # Assign each parameter key value pair to the DataFrame for the corresponding subset
            df.loc[dataframe_indices, key] = value

df.head(10)

## 9. Analysis

In [ ]:
# Visualize how the predator and prey populations change over time

# Notice that the populations are more chaotic when the prey birth rate is higher,
# and the system is more stable when it is lower.

px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['predator_population', 'prey_population'], # Variables on the vertical axis
    line_group='run', # One line for each MC run
    facet_row='prey_birth_parameter', # Create a figure for each `prey_birth_parameter` parameter sweep
    log_y=True, # Use log scale on the vertical axis
    height=800,
)

In [ ]:
# Visualize how the predator and prey populations compare

# Notice that they tend to have cycles around a fixed point
# which means that their populations are never static,
# but rather cyclic with ups and downs.

px.line(
    df,
    x='predator_population', # Variable on the horizontal axis
    y='prey_population', # Variable on the vertical axis
    color='run', # Color by MC run
    facet_row='prey_birth_parameter', # Create a figure for each `prey_birth_parameter` parameter sweep
    log_x=True, # Use log scale on the horizontal axis
    log_y=True, # Use log scale on the vertical axis
    height=800,
)